# Boston House Prices Prediction

# Table of Contents
1. [Environment](#Environment)
2. [Load Data](#Load-Data)
3. [Data Analysis](#Data-Analysis)
4. [Data Preprocessing](#Data-Preprocessing)
5. [Data Visualization](#Data-Visualization)
6. [Feature Selection](#Feature-Selection)
7. [Models Init](#Models-Init)
8. [Models Evaluation](#Models-Evaluation)
9. [GridSearchCV](#GridSearchCV)
10. [Final Ensemble Evaluation](#Fianl-EnsembleEvaluation)

# Environment <a class="anchor" id="Environment"></a>
|Environment|Name|Version|
|-|-|-|
|Run Environment|jupyter notebook|6.5.2|
|python|python3|3.8.8|
|Install method|pip|23.2.1|
|Conda|conda|4.11.0|


## Package Version
|Package|Version|
|-|-|
|numpy|1.23.4|
|pandas|4.11.0|
|scikit-learn|1.0.2|
|seaborn|0.11.0|
|xgboost|2.0.0|
|catboost|1.2.2|
|lightgbm|4.1.0|

In [ ]:
# Import

from sklearn.metrics import mean_squared_error as MSE
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np

In [ ]:
# random
import random
random.seed(1)

# Load Data<a class="anchor" id="Load-Data"></a>

In [ ]:
name= ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
raw_df = pd.read_csv(filepath_or_buffer="../input/boston-house-prices/housing.csv",delim_whitespace=True,names=name)
raw_df.head()

# Data Analysis<a class="anchor" id="Data-Analysis"></a>

Boston House Price dataset has 14 features and their description is given as follows:

- CRIM per capita crime rate by town
- ZN proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS proportion of non-retail business acres per town
- CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- NOX nitric oxides concentration (parts per 10 million)
- RM average number of rooms per dwelling
- AGE proportion of owner-occupied units built prior to 1940
- DIS weighted distances to five Boston employment centres
- RAD index of accessibility to radial highways
- TAX full-value property-tax rate per dollar 10,000.
- PTRATIO pupil-teacher ratio by town
- B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT % lower status of the population
- MEDV Median value of owner-occupied homes in $1000's

# Take a peek at the data

In [ ]:
print(raw_df.shape)
print(raw_df['MEDV'].size)

# Data Preprocessing<a class="anchor" id="Data-Preprocessing"></a>

## Data Imputation
There are not missing value

In [ ]:
df = raw_df
print(df.isnull().sum())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Data Visualization<a class="anchor" id="Data-Visualization"></a>

## Box Plot

In [ ]:
# Plot Design
rc = {
    "axes.facecolor": "#FFF9ED",
    "figure.facecolor": "#FFF9ED",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}

sns.set(rc=rc)

In [ ]:
import plotly.express as px

# plt.figure(figsize=(14, len(df.columns) *2.5))
for i, col in enumerate(df.columns):
    fig = px.box(data_frame=df,y=col,orientation='v', 
                 color_discrete_sequence=['lightblue'], template='ggplot2')
    fig.show()
    
# plt.tight_layout()
# plt.show()

## Histogram Plot

In [ ]:
for i, col in enumerate(df.columns):
    fig = px.histogram(data_frame=df, x=col, nbins=50, 
                       color_discrete_sequence=['#237fb8'], 
                       template= 'ggplot2')
    fig.show()

# Heatmap

In [ ]:
plt.figure(figsize=(16, 9))
corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(df.corr(), mask=mask, annot=True, cmap="YlOrBr")
plt.show()

# Feature Selection<a class="anchor" id="Feature-Selection"></a>

Choose the best performance feature

In [ ]:
X = pd.DataFrame(np.c_[df.RM, df.LSTAT], # Please put features inside the bracket
                 columns = ["RM", "LSTAT"] # Please put feature names here 
                )

target_column = 'MEDV'
corr_filtered = corr.drop('MEDV',axis=0)
# Select Feature
feature_columns1 = corr_filtered[(abs(corr_filtered['MEDV']) >= 0.6)].index
feature_columns2 = corr_filtered[(abs(corr_filtered['MEDV']) >= 0.4)].index
feature_columns3 = corr_filtered[(abs(corr_filtered['MEDV']) >= 0.2)].index

print(feature_columns1)
print(feature_columns2)
print(feature_columns3)

## Using Linear Regression and RandomForestRegressor to test which feature columns is best 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X1 = StandardScaler().fit_transform(pd.DataFrame(df[feature_columns1]))
X2 = StandardScaler().fit_transform(pd.DataFrame(df[feature_columns2]))
X3 = StandardScaler().fit_transform(pd.DataFrame(df[feature_columns3]))
Y = df[target_column]
X = {'feature1':X1, 'feature2':X2, 'feature3':X3}

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

mse_dict = {}
r2_dict = {}
for key, X_df in X.items():
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    mse_list = []
    r2_list = []
    for train_index, test_index in kf.split(X_df,Y):
        X_train, y_train = X_df[train_index], Y[train_index]
        X_test, y_test = X_df[test_index], Y[test_index]
        
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        
        # Prediction
        y_pred = lr.predict(X_test)
        
        # Score and MSE
        mse_list.append(mean_squared_error(y_test, y_pred))
        r2_list.append(r2_score(y_test, y_pred))
        
#     Add to dictionary
    mse_dict[key] = mse_list
    r2_dict[key] = r2_list

In [ ]:
for key, mse_list in mse_dict.items():
    print(f'{key} column: has MSE:{np.mean(mse_list)} R2_Score:{np.mean(r2_dict[key])}')

`feature_columns3` has the best performance in LinearRegressor

In [ ]:
print('Feature3:',feature_columns3)

## RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

mse_dict_rfr = {}
r2_dict_rfr = {}
for key, X_df in X.items():
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    mse_list = []
    r2_list = []
    for train_index, test_index in kf.split(X_df,Y):
        X_train, y_train = X_df[train_index], Y[train_index]
        X_test, y_test = X_df[test_index], Y[test_index]
        
        rfr = RandomForestRegressor(100, max_depth=8, random_state=42)
        rfr.fit(X_train, y_train)
        
        # Prediction
        y_pred = rfr.predict(X_test)
        
        # Score and MSE
        mse_list.append(mean_squared_error(y_test, y_pred))
        r2_list.append(r2_score(y_test, y_pred))
        
#     Add to dictionary
    mse_dict_rfr[key] = mse_list
    r2_dict_rfr[key] = r2_list

In [ ]:
for key, mse_list in mse_dict_rfr.items():
    print(f'{key} column: has MSE:{np.mean(mse_list)} R2_Score:{np.mean(r2_dict_rfr[key])}')


### Using feature3 has the best result in both LinearRegression method or RandomForestRegressor method 

# Models Init <a class="anchor" id="Models-Init"></a>
1. LinearRegression
2. DecisionTree
3. KNearsNeighbors
4. RandomForestRegressor
5. XGBoost
6. CatBoost
7. LGBM
8. Ensemble

In [ ]:
# Import library
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor


In [ ]:
feature_column = feature_columns3

models = {
    'lr' : LinearRegression(),
    'dcr' : DecisionTreeRegressor(),
    'rfr' : RandomForestRegressor(),
    'knr' : KNeighborsRegressor(),
    'xgb' : xgb.XGBRegressor(),
    'cat' : CatBoostRegressor(verbose=0),
    'lgbm' : LGBMRegressor(verbose=0)
}

In [ ]:
from sklearn.model_selection import GridSearchCV

def train_evaluate_model(models, X, y, n_splits):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Create dictionary to store MSE & R2 of each model
    r2_scores = {model: [] for model in models.keys()}
    mse_scores = {model: [] for model in models.keys()}
    # Store best model & fold number in each fold
    best_models = {model: None for model in models.keys()}
    
    for model_name, model in models.items():
        print(f'\033[1;34mTraining {model_name} models\033[0m')  # Blue for model names
        best_score = 0
        for fold, (train_index, test_index) in enumerate(kf.split(X,y)):
            print(f'\033[1;33m######################## Training FOLD {fold+1} ########################\033[0m')  # Yellow for fold numbers
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]

            model.fit(X_train, y_train)

            # Prediction
            y_pred = model.predict(X_test)

            # Score and MSE
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mse_scores[model_name].append(mse)
            r2_scores[model_name].append(r2)
            
            if r2 > best_score:
                best_score = r2
                best_models[model_name] = (model, fold)
                
            print(f'\033[1;35mMSE: {mse:.5f}\033[0m')  # Purple for MSE
            print(f'\033[1;35mR2_Score: {r2:.5f}\033[0m')  # Purple for R2 Score
            
        print(f'\033[1;31m*********************************************\033[0m')  # Red for separator
        print(f'\033[1;35mMean MSE: {np.mean(mse_scores[model_name]):.5f}\033[0m')  # Purple for mean MSE
        print(f'\033[1;35mMean R2_Score: {np.mean(r2_scores[model_name]):.5f}\033[0m')  # Purple for mean R2 Score   
        
    return mse_scores, r2_scores, best_models

In [ ]:
# Ignore warnings from lgbm
# No work !!!
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="[LightGBM] \[Warning\] No further splits with positive gain, best gain: -inf")

X = StandardScaler().fit_transform(pd.DataFrame(df[feature_column]))
y = df[target_column]
mse_scores, r2_scores, best_models = train_evaluate_model(models, X, y, 10)

# Models Evaluation<a class="anchor" id="Models-Evaluation"></a>

In [ ]:
result = pd.DataFrame()
for model_name, score_list in r2_scores.items():
    print(f'{model_name} R2_Score:{np.mean(score_list)} MSE:{np.mean(mse_scores[model_name])}')
    t = pd.DataFrame([[np.mean(score_list), np.std(score_list),
                       np.mean(mse_scores[model_name]), np.std(mse_scores[model_name])]], 
                     columns=['R2_Score','R2_Score_Std', 'MSE', 'MSE_Std'], index=[model_name])
    result = pd.concat([result, t], axis=0)
    
result.style.background_gradient(cmap="YlOrBr")

In [ ]:
plt.figure(figsize=(14, 14*len(best_models)))
for idx, (model_name, tuple_item) in enumerate(best_models.items()):
    model = tuple_item[0]
    ite = tuple_item[1]
    plt.subplot(len(models), 1, idx+1)
    y_pred = model.predict(X)
    plt.scatter(y_pred, Y)
    plt.title(f'Comparison of prediction & true value in {model_name} {ite} fold')
    plt.plot([Y.min(), Y.max()], [Y.min(), Y.max()], 'k--', lw=2)
    plt.xlabel('y_pred')
    plt.ylabel('y_true')
plt.tight_layout()
plt.show()

# GridSearchCV search best hyperparameters<a class="anchor" id="GridSearchCV"></a>

## GridSearchCV CatBoostRegressor

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters_cat = {
    'learning_rate':[0.01, 0.05, 0.1, 0.2],
    'depth':[5, 8, 10],
    'iterations':[30, 50, 100],
    'l2_leaf_reg': [0.0015, 0.002, 0.1]
}

grid_cat = GridSearchCV(estimator=CatBoostRegressor(), param_grid=parameters_cat, cv=5, n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, random_state=42,test_size=0.3)

grid_cat.fit(X_train, y_train)

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid_cat.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_cat.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_cat.best_params_)

In [ ]:
cat = grid.best_estimator_

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X, Y):
    X_train, y_train = X[train_index], Y[train_index]
    X_test, y_test = X[test_index], Y[test_index]
    
    y_pred = cat.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print('R2_score:', r2)
    print('Mean Squared Error:', mse)

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X, Y):
    X_train, y_train = X[train_index], Y[train_index]
    X_test, y_test = X[test_index], Y[test_index]
    
    y_pred = best_models['cat'][0].predict(X_test)

    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print('R2_score:', r2)
    print('Mean Squared Error:', mse)

## GridSearchCV XGBRegressor() 

In [ ]:
parameters_xgbr = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9]
}

grid_xgbr = GridSearchCV(estimator=xgb.XGBRegressor(), param_grid=parameters_xgbr, cv=5, n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, random_state=42, test_size=0.3)

grid_xgbr.fit(X_train, y_train)

xgbr = grid_xgbr.best_estimator_

y_pred = xgbr.predict(X_test)
print(r2_score(y_test, y_pred))


In [ ]:
print(" Results from Grid Search XGBR" )
print("\n The best estimator across ALL searched params:\n", grid_xgbr.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_xgbr.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_xgbr.best_params_)

## GridSearchCV LGBMRegressor

In [ ]:
best_models['lgbm'][0].get_params()

In [ ]:
parameters_lgbm = {
    'task' : ['predict'],
    'boosting': ['gbdt' ],
    'objective': ['root_mean_squared_error'],
    'learning_rate':[ 0.1, 0.05, 0.005 ],
   'num_leaves':[ 7, 15, 31],
   'max_depth' : range(4, 10, 1),
     'n_estimators': [50, 100, 200, 500],
}

grid_lgbm = GridSearchCV(estimator=LGBMRegressor(), param_grid=parameters_lgbm, cv=5, n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, random_state=42, test_size=0.3)

grid_lgbm.fit(X_train, y_train)

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid_lgbm.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_lgbm.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_lgbm.best_params_)

lgbm = grid_lgbm.best_estimator_

# Final Ensemble Evaluation<a class="anchor" id="Fianl-EnsembleEvaluation"></a>

### Create an ensemble model include top three evaluation models {`cat`, `xgb`, `lgbm`} with hyperparameters we search before

In [ ]:
# Todo find the best parame of these three models
from sklearn.ensemble import VotingRegressor

cat_params = {'depth': 5, 'iterations': 100, 'l2_leaf_reg': 0.1, 'learning_rate': 0.1}
xgbr_params = {'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 180}
lgbm_params = {'boosting': 'gbdt', 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'num_leaves': 7, 'objective': 'root_mean_squared_error', 'task': 'predict'}

ensembles = [('cat', CatBoostRegressor(**cat_params)), ('xgbr', xgb.XGBRegressor(**xgbr_params)), ('lgbm', LGBMRegressor(**lgbm_params))]

voting_regressor = VotingRegressor(ensembles)

m, r, mod = train_evaluate_model({'Ensembles':voting_regressor}, X, Y, 10)
mse_scores.update(m)
r2_scores.update(r)
best_models.update(mod)



In [ ]:
r = np.mean(r2_scores['Ensembles'])
mse = np.mean(mse_scores['Ensembles'])
r_std = np.std(r2_scores['Ensembles'])
mse_std = np.std(mse_scores['Ensembles'])

result = pd.concat([result,pd.DataFrame([[r,r_std, mse, mse_std]], index=['Ensebles'], columns=['R2_Score', 'R2_Score_Std', 'MSE', 'MSE_Std'])], axis=0)
result.style.background_gradient(cmap='YlOrBr')

### Plot predict and true value

In [ ]:
y_pred = voting_regressor.predict(X)

plt.figure(figsize=(10,10))
plt.scatter(x=Y, y=y_pred)
plt.xlabel('Y True')
plt.ylabel('Y Predict')
plt.plot([Y.min(), Y.max()], [Y.min(), Y.max()], 'k--', lw=2)
plt.title('Comparison between Prediciton & True value with Ensembles')
plt.show()